In [ ]:
import skimage.restoration as sr
import numpy as np
import glob
# import h5py
import os
import scipy.io as scio
from skimage import exposure
from skimage.io import imsave, imread
from scipy.misc import imresize
from scipy.io import savemat
from scipy import ndimage, misc
import matplotlib.pyplot as plt
%matplotlib inline
import re

from helper import *

import os
cwd = os.getcwd()

In [ ]:
def atoi(text) : 
    return int(text) if text.isdigit() else text

In [ ]:
def natural_keys(text) :
    return [atoi(c) for c in re.split('(\d+)', text)]

### Remove Images in Folders

In [ ]:
remove = False
if remove:
    nlmeans_image_folder_rem = cwd + '/datasets/OCTData/alldata/nl_means_denoised/Train/'
    remove_files(nlmeans_image_folder_rem, '.png')

### Root Folder

Removed folders in whole_data is os.walk will reference them

In [ ]:
root_path = ""
filenames = []

# for root, dirnames, filenames in os.walk("/home/sim/notebooks/relaynet_pytorch/datasets/OCTData/alldata/whole_raw_image/Train"):
for root, dirnames, filenames in os.walk("/Users/sim/Desktop/Imperial/Project/PreTrained/relaynet_pytorch/datasets-24-aug/OCTData/alldata/whole_raw_image"):
    filenames.sort(key = natural_keys)
    rootpath = root
print(len(filenames))

In [ ]:
# print(filenames)
filenames = filenames[1:] # making filenames without .DS_store
# print(filenames)

In [ ]:
def get_info(filenames, ext):
    images = []
    for filename in filenames :
        filepath = os.path.join(root,filename)
        if ext == '.npy':
            image = np.load(filepath)
            h,w = image.shape
            
            if h != 512 or w != 64:
#                 print(h,w) 
                amount = 512 - h
                id_full = np.full((amount, 64), 0)
                image = np.concatenate((image, id_full))
#                 print(image.shape)
        elif ext == '.JPG' or ext == '.tif':
            image = ndimage.imread(filepath, mode = "L")
        images.append(image)
    return images

In [ ]:
raw_images = get_info(filenames, '.tif')

In [ ]:
print (len(raw_images))

In [ ]:
print(filenames)

In [ ]:
denoisedimages = []

In [ ]:
hval = 10
alpha = 15
beta = 1

### Filtering
Apply filters before normalisation - choosing only best normalisation

In [ ]:
# https://docs.opencv.org/3.0-beta/modules/imgproc/doc/filtering.html
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html

# Mean Filtering
def mean_filter(img):
    kernel = np.ones((3,6),np.float32)/25
    denoised = cv2.filter2D(img,-1,kernel)
    return denoised

# Gaussian Filtering for blur
# https://en.wikipedia.org/wiki/Gaussian_blur
def gaussian_filter(img):
    blur=cv2.GaussianBlur(img,(5,5),2)
#     blurred=cv2.cvtColor(blur,cv2.COLOR_BGR2HSV)
    denoised=blur
    return denoised

# bilateralFilter can reduce unwanted noise very well while keeping edges fairly sharp. However, it is very slow compared to most filters.
def bilateral_filter(img): 
    blur=cv2.bilateralFilter(img,5, 65, 65) # choose 5 because takes long time when larger than this, vals of 65,65 as best edge
    denoised=blur
    return denoised

In [ ]:
def denoise_nl_means(newimage):
    '''
    NLM denoised image
    '''
    denoised = sr.denoise_nl_means(newimage, multichannel=False, h=hval)
    denoised = denoised - (alpha*beta)
    denoised[denoised<0]=0
    denoised = denoised.astype(np.uint8)
    return denoised

In [ ]:
mean_filter_images = []
gaussian_filter_images = []
bilateral_filter_images = []
nlm_denoised_images = []
bm3d_denoised_images = []

def denoiseImage(image):
    maxvalue = np.max(image)
    newimage = image*(255.0/maxvalue).astype(np.uint8)
    
    mean_filter_img = mean_filter(image)
    gaussian_filter_img = gaussian_filter(image)
    bilateral_filter_img = bilateral_filter(image)
    nlm_denoised = denoise_nl_means(newimage)
#     bm3d_denoised = denoise_nl_means(newimage)
    
    # Store denoised images in array
    mean_filter_images.append(mean_filter_img)
    gaussian_filter_images.append(gaussian_filter_img)
    bilateral_filter_images.append(bilateral_filter_img)
    nlm_denoised_images.append(nlm_denoised)

In [ ]:
# def bilateral_filter(img, x): 
#     blur=cv2.bilateralFilter(img,5, x, x) # choose 5 because takes long time when larger than this, vals of 25,25 good
# #     denoised=cv2.cvtColor(blur,cv2.COLOR_BGR2HSV)
#     denoised=blur
#     return denoised

# def denoiseImage(image):
#     bilateral_filter_images = []
#     for j in range(15, 75, 5):
#         bilateral_filter_img = bilateral_filter(image, j)
#         bilateral_filter_images.append(bilateral_filter_img)
#     print(len(bilateral_filter_images))
#     return bilateral_filter_images
    
# bil_images = []
# for count, image in enumerate(raw_images):
#     if count < 3:
#         denoisd_image = denoiseImage(image)
#         bil_images.append(denoisd_image)
# print(len(bil_images[0]))

# cols = 3
# rows = 1
# thresh = 3
# # raw Filter

# for index in range(0,3):
#     fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(25,20))
#     for i, ax in enumerate(axes):
#         ax.imshow(raw_images[index],cmap="gray")
#         ax.set_title(filenames[index])

#     # Bilateral Filter
#     fig, axes = plt.subplots(nrows=12, ncols=1, figsize=(100,100))
#     for i, ax in enumerate(axes):
#         ax.imshow(bil_images[index][i],cmap="gray")
#         ax.set_title(filenames[index])

In [ ]:
for count, image in enumerate(raw_images):
    print(count)
    denoiseImage(image)

In [ ]:
# cols = 3
# rows = 1
# thresh = 3
# # raw Filter
# fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(25,20))
# for i, ax in enumerate(axes):
#     if i < thresh:
#         ax.imshow(raw_images[i],cmap="gray")
#         ax.set_title(filenames[i])
        
# # Mean Filter
# fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(25,20))
# for i, ax in enumerate(axes):
#     if i < thresh:
#         ax.imshow(mean_filter_images[i],cmap="gray")
#         ax.set_title(filenames[i])

# # Gaussian Filter
# fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(25,20))
# for i, ax in enumerate(axes):
#     if i < thresh:
#         ax.imshow(gaussian_filter_images[i],cmap="gray")
#         ax.set_title(filenames[i])

# # Bilateral Filter
# fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(25,20))
# for i, ax in enumerate(axes):
#     if i < thresh:
#         ax.imshow(bilateral_filter_images[i],cmap="gray")
#         ax.set_title(filenames[i])

# # Non Local Means Filter
# fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(25,20))
# for i, ax in enumerate(axes):
#     if i < thresh:
#         ax.imshow(nlm_denoised_images[i],cmap="gray")
#         ax.set_title(filenames[i])

In [ ]:
print (np.max(nlm_denoised_images[0]))

In [ ]:
print(len(mean_filter_images))

In [ ]:
print(len(denoisedimages))
directory = '/Users/sim/Desktop/Imperial/Project/PreTrained/relaynet_pytorch/datasets-24-aug/OCTData/alldata/'
for item in range(len(raw_images)):
    name = filenames[item]
    
    mean_file = directory + 'mean_filter/'+ name
    gaussian_file = directory + 'gaussian_filter/'+ name
    bilateral_file = directory + 'bilateral_filter/'+ name
    nl_means_file = directory + 'nl_means_denoised/'+ name
    
    # Saving files
    imsave(mean_file, mean_filter_images[item])
    imsave(gaussian_file, gaussian_filter_images[item])
    imsave(bilateral_file, bilateral_filter_images[item])
    imsave(nl_means_file, nlm_denoised_images[item])